In [1]:
import os
import cv2
import math
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

In [2]:
image_height, image_width = 64,64

In [3]:
sequencelength = 20

In [4]:
data_directory = r"C:\Users\bathu\UCF50"

In [8]:
classes_list = ['BaseballPitch','Basketball','BreastStroke','GolfSwing']

In [9]:
def frames_extraction(video_path):
    frames_list = []
    video_reader = cv2.VideoCapture(video_path)
    frames_in_video = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    frames_to_skip = max(int(frames_in_video / sequencelength), 1)
    for frame_counter in range(sequencelength):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * frames_to_skip)
        success, frame = video_reader.read()
        if not success:
            break
        resized_frame = cv2.resize(frame, (image_height, image_width))
        normalized_frame = resized_frame / 255
        frames_list.append(normalized_frame)
    video_reader.release()
    return frames_list


In [10]:
inputs = []
labels = []
for class_index, class_name in enumerate(classes_list):
    print(f'Extracting Data of {class_name}')
    files_list = os.listdir(os.path.join(data_directory, class_name))
    for file_name in files_list:
        video_file_path = os.path.join(data_directory, class_name, file_name)
        frames = frames_extraction(video_file_path)
        if len(frames) == sequencelength:
            inputs.append(frames)
            labels.append(class_index)

inputs = np.asarray(inputs)
labels = np.array(labels)


Extracting Data of BaseballPitch
Extracting Data of Basketball
Extracting Data of BreastStroke
Extracting Data of GolfSwing


In [11]:
labels = to_categorical(labels)

In [12]:
labels

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32)

In [13]:
inputs_train, inputs_test, labels_train, labels_test = train_test_split(inputs, labels,
                                                                            test_size = 0.25, shuffle = True,
                                                                            random_state = 42)

CONVLSTM

In [14]:
model = Sequential()
model.add(ConvLSTM2D(filters = 4, kernel_size = (3, 3), activation = 'tanh',data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True, input_shape = (sequencelength,
                                                                                      image_height, image_width, 3)))
model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
model.add(TimeDistributed(Dropout(0.2)))
model.add(ConvLSTM2D(filters = 8, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True))
model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
model.add(TimeDistributed(Dropout(0.2)))
model.add(ConvLSTM2D(filters = 14, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
recurrent_dropout=0.2, return_sequences=True))
model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
model.add(TimeDistributed(Dropout(0.2)))
model.add(ConvLSTM2D(filters = 16, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
recurrent_dropout=0.2, return_sequences=True))
model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
model.add(TimeDistributed(Dropout(0.2)))
model.add(Flatten()) 
model.add(Dense(len(classes_list), activation = "softmax"))
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])
 

In [15]:
model.fit(x = inputs_train, y = labels_train, epochs = 50, batch_size = 4,
                                                     shuffle = True, validation_split = 0.2, 
                                                     )

Epoch 1/50
80/80 [==============================] - 248s 3s/step - loss: 1.3124 - accuracy: 0.3533 - val_loss: 1.0188 - val_accuracy: 0.5125
Epoch 2/50
80/80 [==============================] - 140s 2s/step - loss: 0.8794 - accuracy: 0.5836 - val_loss: 0.7664 - val_accuracy: 0.6375
Epoch 3/50
80/80 [==============================] - 71s 895ms/step - loss: 0.6869 - accuracy: 0.7066 - val_loss: 0.5884 - val_accuracy: 0.7125
Epoch 4/50
80/80 [==============================] - 76s 950ms/step - loss: 0.5692 - accuracy: 0.7603 - val_loss: 0.4593 - val_accuracy: 0.7875
Epoch 5/50
80/80 [==============================] - 143s 2s/step - loss: 0.4811 - accuracy: 0.8233 - val_loss: 0.3850 - val_accuracy: 0.7625
Epoch 6/50
80/80 [==============================] - 136s 2s/step - loss: 0.3714 - accuracy: 0.8580 - val_loss: 0.4463 - val_accuracy: 0.8250
Epoch 7/50
80/80 [==============================] - 111s 1s/step - loss: 0.3222 - accuracy: 0.8801 - val_loss: 0.3463 - val_accuracy: 0.8625
Epoch 8/5

In [16]:
model.evaluate(inputs_test, labels_test)

5/5 [==============================] - 6s 997ms/step - loss: 0.5329 - accuracy: 0.9173


[0.5329467058181763, 0.9172932505607605]

LRCN

In [17]:
lrcn = Sequential()
lrcn.add(TimeDistributed(Conv2D(16, (3, 3), padding='same',activation = 'relu'),
                              input_shape = (sequencelength,image_height,image_width, 3)))
lrcn.add(TimeDistributed(MaxPooling2D((4, 4)))) 
lrcn.add(TimeDistributed(Dropout(0.25)))
lrcn.add(TimeDistributed(Conv2D(32, (3, 3), padding='same',activation = 'relu')))
lrcn.add(TimeDistributed(MaxPooling2D((4, 4))))
lrcn.add(TimeDistributed(Dropout(0.25)))
lrcn.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
lrcn.add(TimeDistributed(MaxPooling2D((2, 2))))
lrcn.add(TimeDistributed(Dropout(0.25)))
lrcn.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
lrcn.add(TimeDistributed(MaxPooling2D((2, 2))))
lrcn.add(TimeDistributed(Dropout(0.25)))
lrcn.add(TimeDistributed(Flatten()))
lrcn.add(LSTM(32))
lrcn.add(Dense(len(classes_list), activation = 'softmax'))
lrcn.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])


In [18]:
lrcn.fit(x = inputs_train, y = labels_train, epochs = 50, batch_size = 4,
                                                     shuffle = True, validation_split = 0.2, 
                                                     )

Epoch 1/50
80/80 [==============================] - 12s 124ms/step - loss: 1.3544 - accuracy: 0.3123 - val_loss: 1.2649 - val_accuracy: 0.4625
Epoch 2/50
80/80 [==============================] - 6s 80ms/step - loss: 1.0636 - accuracy: 0.5016 - val_loss: 0.8997 - val_accuracy: 0.4875
Epoch 3/50
80/80 [==============================] - 6s 70ms/step - loss: 0.8951 - accuracy: 0.5741 - val_loss: 0.7883 - val_accuracy: 0.6125
Epoch 4/50
80/80 [==============================] - 4s 55ms/step - loss: 0.7487 - accuracy: 0.6246 - val_loss: 0.7918 - val_accuracy: 0.5875
Epoch 5/50
80/80 [==============================] - 4s 51ms/step - loss: 0.6838 - accuracy: 0.7161 - val_loss: 0.7989 - val_accuracy: 0.5875
Epoch 6/50
80/80 [==============================] - 4s 56ms/step - loss: 0.6234 - accuracy: 0.7319 - val_loss: 0.4793 - val_accuracy: 0.7750
Epoch 7/50
80/80 [==============================] - 4s 55ms/step - loss: 0.5584 - accuracy: 0.7571 - val_loss: 0.4928 - val_accuracy: 0.8125
Epoch 8/50


In [19]:
lrcn.evaluate(inputs_test, labels_test)

5/5 [==============================] - 1s 67ms/step - loss: 0.2679 - accuracy: 0.9549


[0.26785728335380554, 0.9548872113227844]